#### Load necessary libraries

In [ ]:
import os
import librosa
import numpy as np
import pandas as pd
from sklearn.preprocessing import label_binarize
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam
from sklearn import metrics 
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split

#### Filepath for data and some constant

In [281]:
#filepath of file with labels and training set
name = 'C:/Users/Python/Documents/STC_test/data_v_7_stc/meta/meta.txt'
path_training = 'C:/Users/Python/Documents/STC_test/data_v_7_stc/audio/'
#put here filepath for new set
path_test = 'C:/Users/Python/Documents/STC_test/data_v_7_stc/test/' 
n_mfcc = 50
number_fun = 2

#### Reading data

In [282]:
#obtain features and scaling for new set
features_test = np.array([]).reshape(0, n_mfcc*number_fun)
name_test_file = np.array([])
for file in os.listdir(path_test):
    x, sr = librosa.load(os.path.join(path_test,file))
    mfcc = librosa.feature.mfcc(x, sr, n_mfcc = n_mfcc)
    mfcc_mean = np.mean(mfcc.T,axis=0)
    mfcc_median = np.median(mfcc.T,axis=0)
    features_test = np.vstack((features_test, np.concatenate((mfcc_mean, mfcc_median))))
    name_test_file = np.append(name_test_file, file)
test = StandardScaler().fit_transform(features_test)

In [ ]:
#transform and obtain features from training audio file 
features = np.array([]).reshape(0, n_mfcc*number_fun)
name_file = np.array([])
for file in os.listdir(path_training):
    x, sr = librosa.load(os.path.join(path_training,file))
    mfcc = librosa.feature.mfcc(x, sr, n_mfcc = n_mfcc)
    mfcc_mean = np.mean(mfcc.T,axis=0)
    mfcc_median = np.median(mfcc.T,axis=0)
    features = np.vstack((features, np.concatenate((mfcc_mean, mfcc_median))))
    name_file = np.append(name_file, file)

In [283]:
#load label of classes
label = pd.read_csv(name, sep = '\t', header=None)
label.columns = ['name', 'place', 'start', 'end', 'class']
features_data = pd.DataFrame(np.concatenate((name_file.reshape(name_file.shape[0],1), features), axis=1))
features_data.columns = np.concatenate((['name'], ['feature_'+str(i) for i in range(1,n_mfcc*number_fun+1)]))
train = pd.merge(label, features_data, on='name')

ValueError: Length mismatch: Expected axis has 201 elements, new values have 101 elements

#### Classifcation

In [264]:
#scaling training set
X = train.drop(['class','name', 'place', 'start', 'end'], axis = 1)
X = StandardScaler().fit_transform(X)

In [265]:
# Binarize the output
y = label_binarize(train['class'], classes=train['class'].unique())

In [266]:
#split set on train and test
train_X, test_X, train_y, test_y = train_test_split(X,y,train_size=0.5, test_size = 0.5,random_state=40)

In [267]:
num_labels = y.shape[1]

In [268]:
# build model
model = Sequential()

model.add(Dense(32, input_shape=(n_mfcc*number_fun,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(32))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(num_labels))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [275]:
model.fit(train_X, train_y, batch_size=50, epochs=30, validation_data=(test_X, test_y))
scores = model.evaluate(test_X, test_y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

Train on 5653 samples, validate on 5654 samples
Epoch 1/50
5653/5653 [==============================] - 0s 63us/step - loss: 0.1165 - acc: 0.9620 - val_loss: 0.0437 - val_acc: 0.9903
Epoch 2/50
5653/5653 [==============================] - 0s 60us/step - loss: 0.1191 - acc: 0.9641 - val_loss: 0.0429 - val_acc: 0.9901
Epoch 3/50
5653/5653 [==============================] - 0s 59us/step - loss: 0.1011 - acc: 0.9678 - val_loss: 0.0404 - val_acc: 0.9913
Epoch 4/50
5653/5653 [==============================] - 0s 60us/step - loss: 0.1043 - acc: 0.9669 - val_loss: 0.0444 - val_acc: 0.9897
Epoch 5/50
5653/5653 [==============================] - 0s 61us/step - loss: 0.0980 - acc: 0.9699 - val_loss: 0.0456 - val_acc: 0.9901
Epoch 6/50
5653/5653 [==============================] - 0s 58us/step - loss: 0.1183 - acc: 0.9614 - val_loss: 0.0437 - val_acc: 0.9899
Epoch 7/50
5653/5653 [==============================] - 0s 61us/step - loss: 0.1011 - acc: 0.9692 - val_loss: 0.0433 - val_acc: 0.9903
Epoch 8

#### Prediction and results

In [276]:
test_class = model.predict_classes(test)

values = train['class'].unique()
keys = range(num_labels)
dictionary = dict(zip(keys, values))
test_class = [dictionary.get(i) for i in test_class]

In [277]:
test_prob = model.predict(test)

In [278]:
result = pd.DataFrame()

In [279]:
result['file_name'] = name_test_file
result['predict'] = np.max(test_prob, axis = 1)
result['class'] = test_class

In [280]:
result

,file_name,predict,class
0,background_0001.wav,0.683995,background
1,background_0004.wav,0.955480,keyboard
2,background_0009.wav,0.834575,keyboard
3,background_0012.wav,0.568661,keyboard
4,background_0016.wav,0.971830,keyboard
5,background_0018.wav,0.420086,ring
6,background_0022.wav,0.999206,keyboard
7,background_0028.wav,0.999996,keyboard
8,background_0031.wav,0.999998,keyboard
9,background_0036.wav,0.999981,keyboard


In [259]:
result.to_csv('C:/Users/Python/Documents/STC_test/result.txt', sep = '\t', header =False, index = False)